# Module and Global Settings

In [ ]:
import pandas as pd
import numpy as np
import math
import copy
import random
import os

In [ ]:
# SET PATH HERE
path_home = "/PATH/TO/ghlda" 

In [ ]:
train_or_test = "train"
data = "wiki"
date = "20191226-40"
word_embedding = "glove" # glove,word2vec,fasttext
file_corpus = data + "_train_5000.txt"
file_corpus_test = data + "_test_1000.txt"

path_file =  path_home + "/data/" + data + "/"

# Specify word embedding type and path
path_emb = path_home + "/data/" 

# Specify cpp path
path_cpp = path_home + "/cpp"

# Specify save path
path_save = path_home + "/results/" + data + "/"

# Parameter

In [ ]:
num_topics = 40
mult_num_list = [50,40,30,20]
nu_add = 0.1
mult_num = mult_num_list[0]
kappa = 0.1
alpha_init = 0.1
word_thres = 49 # above this value

# DO NOT CHANGE
method = "cgtm"
result_file = "result" + method + date + ".txt"

# Sepcify file header
file_header = file_corpus.split(".")[0] + "_" + word_embedding + "_" + method + "_" + date
print("file name starts with:" + file_header)

# remove existing result
if train_or_test == "train":
    current = os.getcwd()
    os.chdir(path_save)
    print(path_save)
    all_files = os.listdir()
    hantei = 0
    try:
        for i in range(len(all_files)):
            file_t = all_files[i]
            if (file_t != "old") & (file_t != "FIN"):
                #print(file_t)
                v1 = file_t.split("_")
                if (v1[0] == data) & (v1[2] == version) & (v1[4] == word_embedding) & (v1[5] == method) & (v1[6] == date):
                    os.remove(file_t)
                    print("removed: " + file_t)
                    hantei = 1
    except:
        pass
    if hantei == 0:
        print("removed none")
    os.chdir(current)

# Load word embedding

In [ ]:
%%time
type2word2vec = {}
if train_or_test == "train":
    f = open(path_emb + "glove.6B.50d.txt", "r")
    vectors = f.readlines()
    word2vec = {}
    for vec in vectors:
        vec_list = vec.strip().split(" ")
        temp_vec = np.asarray(vec_list[1:]).astype("float32")
        word2vec.update({vec_list[0]:temp_vec}) 
    type2word2vec.update({"glove":word2vec})
    print("Finished Loading glove")
    word2vec = {}
    import gensim
    from gensim.models import KeyedVectors
    word2vec = KeyedVectors.load_word2vec_format(path_emb  + 'GoogleNews-vectors-negative300.bin', 
                                              binary=True)
    type2word2vec.update({"word2vec":word2vec})
    print("Finished Loading word2vec")
    f = open(path_emb + "wiki-news-300d-1M.vec", "r")
    vectors = f.readlines()
    word2vec = {}
    for i in range(1,len(vectors)):
        vec_list = vectors[i].strip().split(" ")
        temp_vec = np.asarray(vec_list[1:]).astype("float32")
        word2vec.update({vec_list[0]:temp_vec})   
    type2word2vec.update({"fasttext":word2vec})
    print("Finished Loading fasttext")
    print("We are using " + word_embedding)
    word2vec = type2word2vec[word_embedding]
    embedding_dimension = word2vec["the"].shape[0]
    
else:
    if word_embedding == "fasttext":
        f = open(path_emb  + "wiki-news-300d-1M.vec", "r")
        vectors = f.readlines()
        word2vec = {}
        for i in range(1,len(vectors)):
            vec_list = vectors[i].strip().split(" ")
            temp_vec = np.asarray(vec_list[1:]).astype("float32")
            word2vec.update({vec_list[0]:temp_vec})   
        type2word2vec.update({"fasttext":word2vec})
        print("Finished Loading fasttext")
        print("We are using " + word_embedding)
        embedding_dimension = word2vec["the"].shape[0]    
    
    elif word_embedding == "word2vec":
        word2vec = {}
        import gensim
        from gensim.models import KeyedVectors
        word2vec = KeyedVectors.load_word2vec_format(path_emb  + 'GoogleNews-vectors-negative300.bin', 
                                                  binary=True)
        type2word2vec.update({"word2vec":word2vec})
        print("Finished Loading word2vec")
        print("We are using " + word_embedding)
        embedding_dimension = word2vec["the"].shape[0]
        
    else:
        f = open(path_emb + "glove.6B.50d.txt", "r")
        vectors = f.readlines()
        word2vec = {}
        for vec in vectors:
            vec_list = vec.strip().split(" ")
            temp_vec = np.asarray(vec_list[1:]).astype("float32")
            word2vec.update({vec_list[0]:temp_vec}) 
        type2word2vec.update({"glove":word2vec})
        print("Finished Loading glove")
        print("We are using " + word_embedding)
        embedding_dimension = word2vec["the"].shape[0]
        
f = open(path_file + file_corpus, "r")
corpus_data = f.readlines()


file_temp = path_emb + "/" + data + "/" + data + "_word2cnt.csv"
df_word2cnt = pd.read_csv(file_temp)
word2cnt = {}
for i in range(len(df_word2cnt)):
    word2cnt.update({df_word2cnt["word"].iloc[i]:df_word2cnt["cnt"].iloc[i]})

# Read file

In [ ]:
df_word_0 = pd.DataFrame(pd.Series(word2cnt))
df_word_0.reset_index(inplace=True)
df_word_0.columns=["word","cnt"]
cond = df_word_0["cnt"] > word_thres
df_word = df_word_0.loc[cond]
print(np.sum(cond))
print(np.sum(~cond))

word2cnt_thres = {}
for i in range(len(df_word)):
    word2cnt_thres.update({df_word["word"].iloc[i]:df_word["cnt"].iloc[i]})
    
doc_word_topic_list = []
id2word, word2id, doc2cnt = ({} for _ in range(3))
id_cnt = 0
embedding_agg = 0 * word2vec["the"]
for index, doc in enumerate(corpus_data):  
    temp = doc.strip().split(' ')
    for word in temp:
        if word in word2cnt_thres:
            if word in word2id:
                word_id = word2id[word]
            else:
                word2id.update({word: id_cnt})
                id2word.update({id_cnt: word})
                embedding_agg += word2vec[word]
                word_id = id_cnt
                id_cnt += 1
            if int(index) in doc2cnt:
                doc2cnt[int(index)] += 1
            else:
                doc2cnt.update({int(index):1})
                
            temp_vec = word2vec[word]
            doc_word_topic_list.append([int(index),int(word_id),
                                        random.randint(0,num_topics-1),
                                        0,0,0,0,0])
doc_word_topic = np.asarray(doc_word_topic_list)
num_docs = max(doc_word_topic[:,0]) + 1        
   
word2cdf = {}
df_word_2 = df_word.sort_values(by="cnt",ascending=False)
df_word_2.reset_index(inplace=True)
df_word_2.columns=["ignore","word","cnt"]
agg = np.sum(df_word_2["cnt"])
temp_agg = 0.0
for i in range(len(df_word_2)):
    temp_agg += float(df_word_2.loc[df_word_2.index==i,"cnt"])
    df_word_2.loc[df_word_2.index==i,"cnt"]  = temp_agg / agg
    word = int(word2id[df_word_2["word"].iloc[i]])
    cnt = float(df_word_2["cnt"].iloc[i])
    word2cdf.update({word:cnt}) 

embedding_matrix_list = []            
embedding_center = embedding_agg / len(word2id)
embedding_center = np.reshape(embedding_center,[1,-1])
embedding_covariance = np.zeros(shape=(embedding_dimension,embedding_dimension))
for i in range(len(word2id)):
    word = id2word[i]
    temp_vec = word2vec[word] 
    embedding_matrix_list.append(temp_vec)
    embedding_covariance += (temp_vec - embedding_center).transpose() *  (temp_vec - embedding_center)
embedding_matrix = np.asarray(embedding_matrix_list)
normalized_covariance = embedding_covariance / len(word2id)

df_doc = pd.DataFrame(pd.Series(doc2cnt))
df_doc.reset_index(inplace=True)
df_doc.columns = ["doc","cnt"]
df_doc.sort_values(by="cnt",ascending=True,inplace=True)
print(df_doc.head())
print(doc_word_topic.shape)

# Import GHLDA Module

In [ ]:
os.chdir(path_cpp)

In [ ]:
%%bash
make

In [ ]:
import ghlda
GHLDA = ghlda.GHLDA("")

Psi =  mult_num * np.identity(embedding_center.shape[1])
GHLDA.num_topics = num_topics
GHLDA.Psi = Psi
nu = embedding_matrix.shape[1]  + nu_add
GHLDA.nu = nu
GHLDA.kappa = kappa 
GHLDA.embedding_dimension = embedding_dimension
GHLDA.alpha_topic_vec = np.zeros(num_topics) + alpha_init

# Pass data
GHLDA.embedding_matrix = embedding_matrix
GHLDA.embedding_center = embedding_center

# Create initial assignment
GHLDA.doc_word_topic = doc_word_topic
GHLDA.create_doc2place_vec()
GHLDA.glda_calc_tables_parameters_from_assign()
GHLDA.zero_mu = 0*GHLDA.topic2mu[0]
GHLDA.zero_Sigma = 0 * GHLDA.topic2Sigma[0]

# Run settings

In [ ]:
import codecs
verbose = 0
epocs_you_want = 1
num_thread = 14
parallel_loop = num_thread
total_inner = 100
total = math.floor(epocs_you_want * GHLDA.num_docs / (parallel_loop * total_inner))+1
GHLDA.min_diagonal_val = 0.000001
base = math.floor(math.log10(np.max(GHLDA.topic2Sigma[0])))
if base > 0:
    if base % 2 != 0:
        base -= 1
    GHLDA.shrink = 1 / np.power(10,base)
else:
    GHLDA.shrink = 1.0
print(GHLDA.Psi)

GHLDA.initial_eta_vec = np.zeros((num_topics, 1))
GHLDA.initial_lambda_vec = np.zeros((num_topics, 1))
GHLDA.cgtm_decide_initial_variable()
GHLDA.cgtm_update_mu_c_Sigma_c(1)

# Run (Use only when training)

In [ ]:
%%time
if train_or_test == "train":    
    saved_epoc_int = {}
    for itr in range(total):
        num_sampled = (itr+1) * parallel_loop*total_inner
        epoc = int(num_sampled / num_docs)
        GHLDA.cgtm_collapsed_gibbs_sample_parallel_for_all(total_inner, num_thread, 
                                                           num_thread, 0, 0)
        if itr % 50 == 0:
            GHLDA.glda_calc_tables_parameters_from_assign()
            print("iteration:" + str(itr) + ", epoc:" + str(num_sampled / GHLDA.num_docs))
        if (epoc > 0.9) &(epoc not in saved_epoc_int):
            saved_epoc_int.update({epoc:1})
            GHLDA.make_Eigen_doc2eta_lambda_vec()            
            file_out = path_save + file_header + "_epoc_" + str(epoc) 
            np.savez_compressed(file_out, 
                                doc_word_topic = GHLDA.doc_word_topic,
                                word_thres = word_thres,
                                num_topics = num_topics,
                                mult_num = mult_num,
                                Psi = GHLDA.Psi,
                                nu = nu,
                                kappa = kappa,
                                alpha_topic_vec = GHLDA.alpha_topic_vec,
                                mu_c = GHLDA.mu_c,
                                Sigma_c = GHLDA.Sigma_c,
                                doc2eta_table = GHLDA.doc2eta_table,
                                doc2lambda_table = GHLDA.doc2lambda_table)
            print("saved:" + file_out)
            print("num_topics:" + str(len(set(GHLDA.doc_word_topic[:,2]))))
            print(GHLDA.topic2num)
            df_topic_0 = pd.DataFrame({"est":GHLDA.doc_word_topic[:,2],
                                       "word":GHLDA.doc_word_topic[:,1]})
            df_topic = pd.DataFrame(df_topic_0.groupby(["est","word"]).size())
            df_topic.reset_index(inplace=True)
            df_topic.columns = ["est","word","cnt"]
            df_topic.sort_values(by="cnt",ascending=False,inplace=True)

            for i in range(num_topics):
                cond = df_topic["est"] == i
                df_temp = df_topic.loc[cond]
                num_show = 15
                if(len(df_temp)>0):
                    if len(df_temp) < num_show:
                        num_show = len(df_temp)
                    temp = ""
                    for j in range(num_show):
                        temp += id2word[df_temp["word"].iloc[j]] + ","
                    print("topic " + str(i) + ":" + temp)
            print("--------------------")

# Parse Test Document

In [ ]:
f = open(path_file + file_corpus_test, "r")
corpus_data_test = f.readlines()

word2cnt_test = {}
for index, doc in enumerate(corpus_data_test):  
    temp = doc.strip().split(' ')
    for word in temp:    
        if word in word2vec:
            if word in word2cnt_test:
                word2cnt_test[word] += 1
            else:
                word2cnt_test[word] = 1
                
df_word_0 = pd.DataFrame(pd.Series(word2cnt_test))
df_word_0.reset_index(inplace=True)
df_word_0.columns=["word","cnt"]

cond = df_word_0["cnt"] > word_thres / 5
print(np.sum(cond))
print(np.sum(~cond))

df_word = df_word_0.loc[cond]

word2cnt_thres_test = {}
for i in range(len(df_word)):
    word2cnt_thres_test.update({df_word["word"].iloc[i]:df_word["cnt"].iloc[i]})
    
f = open(path_file + file_corpus_test, "r")
corpus_data_test = f.readlines()

doc_word_topic_test_list = []
doc2cnt_test = {}
id2cnt = len(id2word)
for index, doc in enumerate(corpus_data_test):  
    temp = doc.strip().split(' ')
    for word in temp:    
        if word in word2vec:
            if word in word2cnt_thres:
                if word in word2id:
                    word_id = word2id[word]
                else:
                    word2id.update({word: id_cnt})
                    id2word.update({id_cnt: word})
                    embedding_agg += word2vec[word]
                    word_id = id_cnt
                    id_cnt += 1

                if int(index) in doc2cnt_test:
                    doc2cnt_test[int(index)] += 1
                else:
                    doc2cnt_test.update({int(index):1})
                temp_vec = word2vec[word]
                doc_word_topic_test_list.append([int(index),int(word_id),
                                                0,0,0,0,0,0])
doc_word_topic_test = np.asarray(doc_word_topic_test_list)
num_docs_test = max(doc_word_topic_test[:,0]) + 1    

# Update embedding_matrix
for i in range(len(word2id)):
    word = id2word[i]
    temp_vec = word2vec[word]
    embedding_matrix_list.append(temp_vec)
embedding_matrix_test = np.asarray(embedding_matrix_list)

df_doc = pd.DataFrame(pd.Series(doc2cnt_test))
df_doc.reset_index(inplace=True)
df_doc.columns = ["doc","cnt"]
df_doc.sort_values(by="cnt",ascending=True,inplace=True)
df_doc.head()

# Heldout Likelihood

In [ ]:
%%time
doc_test_ids = list(set(doc_word_topic_test[:,0]))
GHLDA.doc_word_topic_test = doc_word_topic_test
GHLDA.create_doc2place_vec_test()
GHLDA.embedding_matrix = embedding_matrix
GHLDA.id2word = id2word
GHLDA.beta = 0.1
GHLDA.voc = len(id2word)
GHLDA.cgtm_decide_initial_variable_test()

num_R = 10
burn_in = 0
eta_thres = 0.1
approx = 1
num_threads = 72
GHLDA.cgtm_evaluate_held_out_log_likelihood(
    doc_test_ids,num_R,burn_in,eta_thres,approx,num_threads,0)

heldout_prob = []
for i in range(len(doc_test_ids)):
    heldout_prob.append(np.sum(np.log(GHLDA.doc_test2heldout_prob[doc_test_ids[i]])))
    
import random
# configure bootstrap
n_iterations = 10000
n_size = int(len(heldout_prob) * 1.0)
# run bootstrap
stats = list()
for i in range(n_iterations):
    # prepare train and test sets
    train = random.choices(heldout_prob, k= n_size)
    stats.append(np.mean(train))
    
lower = np.quantile(stats,0.025)
upper = np.quantile(stats,0.975)

print("num_topics:" +str(num_topics) + ",held-out log-likelihood:" + str(np.mean(heldout_prob)) + "[" + \
     str(lower) + "," + str(upper) + "]")